In [5]:
pip install Keras


     |████████████████████████████████| 378kB 1.4MB/s eta 0:00:01    |█▊                              | 20kB 853kB/s eta 0:00:01
     |████████████████████████████████| 51kB 3.2MB/s eta 0:00:01
     |████████████████████████████████| 51kB 3.0MB/s eta 0:00:01
     |████████████████████████████████| 28.8MB 2.9MB/s eta 0:00:01
     |████████████████████████████████| 15.2MB 2.9MB/s eta 0:00:01
     |████████████████████████████████| 3.0MB 3.1MB/s eta 0:00:01
     |████████████████████████████████| 276kB 2.7MB/s eta 0:00:01
  Running setup.py install for pyyaml ... done
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [19]:
pip install torch

     |████████████████████████████████| 80.6MB 2.9MB/s eta 0:00:01     |██████▌                         | 16.3MB 2.6MB/s eta 0:00:26
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [36]:
pip install seqeval

  Created wheel for seqeval: filename=seqeval-0.0.12-cp38-none-any.whl size=7423 sha256=0e86cc1393097eddb4a21b38097bbf176b848a00dda75c3e738c844e530b67c9
  Stored in directory: /Users/maida/Library/Caches/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval
You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [37]:
from math import nan
from future.utils import iteritems
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
#from utils import get_data, generate_output, guess_human, seed_sequence, get_embeddings, find_closest
import numpy as np
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
import keras as k
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report


In [2]:
data = pd.read_csv("ner_dataset.csv", encoding= 'unicode_escape') 
data.head(672)

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
...,...,...,...,...
667,NaN,Muslim,NNP,B-org
668,NaN,Brotherhood,NNP,I-org
669,NaN,as,IN,O
670,NaN,parts,NNS,O


In [3]:
class SentenceGetter(object):
    
    def __init__(self, dataset):
        self.n_sent = 1
        self.dataset = dataset
        self.empty = False
        agg_func = lambda s: [(w, t) for w,t in zip(s["Word"].values.tolist(),
                                                        s["Tag"].values.tolist())]
        self.grouped = self.dataset.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
          
getter = SentenceGetter(data)
sentences = getter.sentences

In [4]:
words = list(set(data["Word"].values))
nrofWords = len(words)

ner_tags = []
for tag in set(data["Tag"].values):
    if tag is nan or isinstance(tag, float):
        ner_tags.append('unk')
    else:
        ner_tags.append(tag)
nrofTags = len(ner_tags)

In [33]:
index_word = {w: i for i, w in enumerate(words)}
index_tag = {t: i for i, t in enumerate(ner_tags)}
idx2tag = {v: k for k, v in iteritems(index_tag)}

In [58]:
maxlen = max([len(s) for s in sentences])
word_embedding_size = 150

X = [[index_word[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=maxlen, sequences=X, padding="post",value=nrofWords - 1)

y = [[index_tag[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=maxlen, sequences=y, padding="post", value=index_tag["O"])
y = [to_categorical(i, num_classes=nrofTags) for i in y]

# Split train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X[2:3] )
print(y[2:3])
#y_train= np.reshape(2, 17)
#y_test = np.reshape(2, 17)

[[273]]
[array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0.]], dtype=float32)]


In [46]:
input = Input(shape=(maxlen,))
input_dim= nrofWords

lstm_layer = tf.keras.layers.RNN(
        tf.keras.layers.LSTMCell(units=64),
        input_shape=(None, input_dim))

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=nrofWords, output_dim=word_embedding_size, input_length=maxlen),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(lstm_layer),
    tf.keras.layers.Dense(nrofTags, activation='relu'),
    tf.keras.layers.Dropout(0.5),
])

In [47]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [48]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 1, 150)            5276700   
_________________________________________________________________
bidirectional_6 (Bidirection (None, 1, 128)            110080    
_________________________________________________________________
bidirectional_7 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense_3 (Dense)              (None, 17)                2193      
_________________________________________________________________
dropout_3 (Dropout)          (None, 17)                0         
Total params: 5,487,789
Trainable params: 5,487,789
Non-trainable params: 0
_________________________________________________________________


In [49]:
history = model.fit(X_train, np.array(y_train), batch_size=2, epochs=5,
                    validation_split=0.1, verbose=1)


Epoch 1/5


ValueError: in user code:

    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:532 train_step  **
  

In [ ]:
test_pred = model.predict(X_te, verbose=1)

idx2tag = {i: w for w, i in tag2idx.items()}

def pred2label(pred):
    out = []
    for pred_i in pred:
        out_i = []
        for p in pred_i:
            p_i = np.argmax(p)
            out_i.append(idx2tag[p_i].replace("PAD", "O"))
        out.append(out_i)
    return out
    
pred_labels = pred2label(test_pred)
test_labels = pred2label(y_te)


In [ ]:
print("F1-score: {:.1%}".format(f1_score(test_labels, pred_labels)))
print(classification_report(test_labels, pred_labels))
